# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd

In [2]:
tweets = pd.read_csv('sentiment140.csv')
#Getting a sample of 20K tweets 
tweets = tweets.sample(n=20000, ignore_index = True)
display(tweets.shape)
tweets.head()

(20000, 6)

,target,ids,date,flag,user,text
0,0,2204071305,Wed Jun 17 01:14:55 PDT 2009,NO_QUERY,leamcfly17,@tommcfly havent replied me in a while! haha! ...
1,4,1997389390,Mon Jun 01 16:45:53 PDT 2009,NO_QUERY,Tomalom,@kbvolley5 Hells yeah! I celebrated for lunch.
2,4,2069212668,Sun Jun 07 15:28:06 PDT 2009,NO_QUERY,schneiderk,@Heather_Poole @keithjjones @flygirl737700 tha...
3,0,2302735956,Tue Jun 23 17:13:25 PDT 2009,NO_QUERY,AshleyStoryline,just wants all the puppies in the world to fin...
4,4,2186340679,Mon Jun 15 18:42:01 PDT 2009,NO_QUERY,redtogray,Getting shit done before the bonfire tonight! ...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
#Importing functions from the first lab
import functions
from functions import *
#Creating an empty column
tweets["text_processed"] = ''

#Aplying functions to all rows
for i in range(len(tweets)):
    tweet_cleaned = clean_up(tweets.text[i])
    tweet_cleaned = tokenize(tweet_cleaned)
    st_lem = stem_and_lemmatize(tweet_cleaned)
    tweets["text_processed"][i] = remove_stopwords(st_lem)

tweets.head()

/var/folders/cv/t2rhm3mj23j8jm825rqyn7bm0000gn/T/ipykernel_17085/960073922.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["text_processed"][i] = remove_stopwords(st_lem)


,target,ids,date,flag,user,text,text_processed
0,0,2204071305,Wed Jun 17 01:14:55 PDT 2009,NO_QUERY,leamcfly17,@tommcfly havent replied me in a while! haha! ...,"[tommcfli, havent, repli, haha, sad, time, wan..."
1,4,1997389390,Mon Jun 01 16:45:53 PDT 2009,NO_QUERY,Tomalom,@kbvolley5 Hells yeah! I celebrated for lunch.,"[kbvolley, hell, yeah, celebr, lunch]"
2,4,2069212668,Sun Jun 07 15:28:06 PDT 2009,NO_QUERY,schneiderk,@Heather_Poole @keithjjones @flygirl737700 tha...,"[heather_pool, keithjjon, flygirl, thank, foll..."
3,0,2302735956,Tue Jun 23 17:13:25 PDT 2009,NO_QUERY,AshleyStoryline,just wants all the puppies in the world to fin...,"[puppi, world, find, good, home, sad, alon, to..."
4,4,2186340679,Mon Jun 15 18:42:01 PDT 2009,NO_QUERY,redtogray,Getting shit done before the bonfire tonight! ...,"[get, shit, done, befor, bonfir, tonight, got,..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [4]:
all_words = tweets['text_processed'].sum()

In [5]:
#I found some help here: https://www.quora.com/How-can-I-display-the-output-of-NLTKs-FreqDist-function-in-a-table-format
from nltk.probability import FreqDist
freq = FreqDist(all_words) 
 
all_words_ordered = pd.DataFrame(list(freq.items()), columns = ["Word","Frequency"]) 
#Ordering them by frequency
all_words_ordered = all_words_ordered.sort_values('Frequency')

In [6]:
#Getting those 5000 which appear more
most_comon = all_words_ordered.iloc [-5000:]
most_comon.tail()

,Word,Frequency
29,good,1195
57,wa,1299
66,day,1367
34,get,1450
69,go,1688


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

In [7]:
#Getting the 5000 words that appear most
all_words = FreqDist(all_words)
word_features = list(all_words.keys())[:5000]

In [8]:
def find_features(words):
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [9]:
#Creating a list will all the words from each row and the target
def get_featlist(tweets):
    featlist = []
    for i in range(len(tweets)):
        feat1 = (find_features(tweets.text_processed[i]))
        #If target is 4 = True and if it's 0 = False
        feat2 = (tweets.target[i] == 4)
        featlist.append((feat1, feat2))
    return featlist

In [10]:
featlist = get_featlist(tweets)

In [11]:
featlist[:3]

[({'tommcfli': True,
   'havent': True,
   'repli': True,
   'haha': True,
   'sad': True,
   'time': True,
   'wan': True,
   'follow': True,
   'missionmcflyoz': True,
   'theyz': True,
   'workin': True,
   'hard': True,
   'make': True,
   'big': True,
   'xx': True,
   'kbvolley': False,
   'hell': False,
   'yeah': False,
   'celebr': False,
   'lunch': False,
   'heather_pool': False,
   'keithjjon': False,
   'flygirl': False,
   'thank': False,
   'friday': False,
   'recommend': False,
   'puppi': False,
   'world': False,
   'find': False,
   'good': False,
   'home': False,
   'alon': False,
   'tortur': False,
   'euthan': False,
   'get': False,
   'shit': False,
   'done': False,
   'befor': False,
   'bonfir': False,
   'tonight': False,
   'got': False,
   'firewood': False,
   'clean': False,
   'fun': False,
   'pretti': False,
   'flower': False,
   'hair': False,
   'video': False,
   'live': False,
   'wait': False,
   'better': False,
   'qualiti': False,
   'tho

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

In [12]:
len(featlist)

20000

In [13]:
#Splitting the featlist (training set -> 75%, test set -> 25%)
training_set = featlist[:(int((len(featlist)*0.75)))]
testing_set = featlist[:(int((len(featlist)*0.25)))]

In [14]:
import nltk
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [15]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 82.56


In [16]:
classifier.show_most_informative_features(15)

Most Informative Features
                    hurt = True            False : True   =     25.4 : 1.0
                  cancel = True            False : True   =     23.2 : 1.0
                 headach = True            False : True   =     22.8 : 1.0
                     bum = True            False : True   =     17.2 : 1.0
                    wors = True            False : True   =     16.5 : 1.0
                     sad = True            False : True   =     14.1 : 1.0
                   tummi = True            False : True   =     13.8 : 1.0
                  welcom = True             True : False  =     13.4 : 1.0
                     ach = True            False : True   =     13.1 : 1.0
                    sore = True            False : True   =     13.1 : 1.0
                 appreci = True             True : False  =     12.9 : 1.0
                 depress = True            False : True   =     12.3 : 1.0
                     www = True             True : False  =     12.2 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [17]:
print("Classifier accuracy percent on test:",(nltk.classify.accuracy(classifier, training_set))*100)

Classifier accuracy percent on test: 80.17999999999999


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [18]:
#Trying with less words
n_words = [250, 500, 1000, 2000, 2500, 3000]
for n in n_words:
    print("Number of words:", n)
    word_features = list(all_words.keys())[:n]
    featlist = get_featlist(tweets)
    training_set = featlist[:(int((len(featlist)*0.75)))]
    testing_set = featlist[:(int((len(featlist)*0.25)))]
    classifier = nltk.NaiveBayesClassifier.train(training_set)
    print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
    print("Classifier accuracy percent on test:",(nltk.classify.accuracy(classifier, training_set))*100)

Number of words: 250
Classifier accuracy percent: 66.3
Classifier accuracy percent on test: 65.84
Number of words: 500
Classifier accuracy percent: 69.82000000000001
Classifier accuracy percent on test: 69.44
Number of words: 1000
Classifier accuracy percent: 72.76
Classifier accuracy percent on test: 72.79333333333334
Number of words: 2000
Classifier accuracy percent: 76.25999999999999
Classifier accuracy percent on test: 75.94
Number of words: 2500
Classifier accuracy percent: 77.75999999999999
Classifier accuracy percent on test: 77.11333333333333
Number of words: 3000
Classifier accuracy percent: 78.56
Classifier accuracy percent on test: 77.71333333333334


As I can see, the model obtains a 70% accuracy with only 500 words

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [19]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [20]:
# your code here